# Services (hour)

In [19]:
import pandas as pd

## Database connection

In [20]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [21]:
# Load dimensions
time_dimension = pd.read_sql_table("TIME_DIMENSION", OLAP_connection)
courier_dimension = pd.read_sql_table("COURIER_DIMENSION", OLAP_connection)
customer_dimension = pd.read_sql_table("CUSTOMER_DIMENSION", OLAP_connection)
office_dimension = pd.read_sql_table("OFFICE_DIMENSION", OLAP_connection)
service_status_dimension = pd.read_sql_table("SERVICE_STATUS_DIMENSION", OLAP_connection)

In [22]:
# Load fact table data from OLTP
services = pd.read_sql_table("mensajeria_servicio", OLTP_connection)
service_statuses = pd.read_sql_table("mensajeria_estadosservicio", OLTP_connection)
users_information = pd.read_sql_table("clientes_usuarioaquitoy", OLTP_connection)

## Transformation

In [23]:
# Process service statuses to get latest status per service
# Convert 'hora' from datetime.time to string to avoid TypeError
service_statuses["hora_str"] = service_statuses["hora"].astype(str)

In [24]:
# Combine 'fecha' and 'hora_str' into a single datetime column
service_statuses["datetime"] = pd.to_datetime(
		service_statuses["fecha"]
) + pd.to_timedelta(service_statuses["hora_str"])

In [25]:
# Sort by 'datetime' and drop duplicates to get the latest status per 'servicio_id'
latest_status = service_statuses.sort_values("datetime").drop_duplicates(
		"servicio_id", keep="last"
)

In [26]:
# Rename columns for merging
latest_status = latest_status.rename(
	columns={
				"id": "status_record_id",
				"estado_id": "status_id",
				"servicio_id": "service_id"
	}
)

In [27]:
# Rename columns in services for consistency
services = services.rename(
		columns={
				"id": "service_id",
				"cliente_id": "customer_id",
				"mensajero_id": "courier_id"
		}
)

In [28]:
null_couriers = services["courier_id"].isna().sum()
print(f"Number of services with no assigned courier: {null_couriers}")

Number of services with no assigned courier: 727


In [29]:
# Ignore services with no assigned courier
services = services.dropna(subset=["courier_id"])

In [30]:
# Merge the latest status with services
services = services.merge(
		latest_status[["service_id", "status_id"]], on="service_id", how="left"
)

In [31]:
# Print number of services with no status
null_status = services["status_id"].isna().sum()
print(f"Number of services with no status: {null_status}")

Number of services with no status: 0


In [32]:
# Combine 'fecha_solicitud' and 'hora_solicitud' into 'request_time'
services["hora_solicitud_str"] = services["hora_solicitud"].astype(str)
services["request_time"] = pd.to_datetime(
		services["fecha_solicitud"]
) + pd.to_timedelta(services["hora_solicitud_str"])
services["request_time"] = services["request_time"].dt.floor("h")

In [33]:
# Create a mapping from datetime to time_id
# Ensure that 'time_dimension["date"]' is in datetime format
time_dimension["date"] = pd.to_datetime(time_dimension["date"])
time_mapping = dict(zip(time_dimension["date"], time_dimension["time_id"]))

In [34]:
# Map 'request_time' to 'time_id'
services["time_id"] = services["request_time"].map(time_mapping)

In [35]:
# Check for unmapped 'time_id's
unmapped_times = services["time_id"].isna().sum()
if unmapped_times > 0:
	print(f"Warning: {unmapped_times} records have unmapped 'request_time'.")

In [42]:
# Select necessary columns for the fact table
service_fact = services[
		[
				"service_id",
				"customer_id",
				"courier_id",
				"status_id",
				"usuario_id",
				"time_id",
		]
]

In [43]:
# Aggregate to create 'total_services' by grouping
service_fact = (
		service_fact.groupby(["time_id", "customer_id", "courier_id", "usuario_id"])
		.agg(total_services=pd.NamedAgg(column="service_id", aggfunc="count"))
		.reset_index()
)

In [46]:
null_couriers = service_fact["courier_id"].isna().sum()
print(f"Number of records with 'courier_id' empty: {null_couriers}")

Number of records with 'courier_id' empty: 0


In [47]:
service_fact = service_fact.merge(courier_dimension, left_on="courier_id", right_on="original_courier_id", how="inner")
service_fact.drop(columns=["courier_id_x", "courier_id_y", "courier_city"], inplace=True)
service_fact.rename(columns={ "original_courier_id": "courier_id" }, inplace=True)

In [48]:
service_fact = service_fact.merge(users_information[["id", "sede_id"]], left_on="usuario_id", right_on="id", how="inner")
service_fact.drop(columns=["usuario_id", "id"], inplace=True)

In [49]:
service_fact = service_fact.merge(office_dimension[["office_id", "original_office_id"]], left_on="sede_id", right_on="original_office_id", how="inner")
service_fact.drop(columns=["sede_id", "original_office_id"], inplace=True)

In [50]:
service_fact.reset_index(inplace=True)
service_fact = service_fact.rename(columns={ "index": "service_hour_id" })
service_fact.set_index("service_hour_id", inplace=True)

## Load

In [51]:
from sqlalchemy import BigInteger


dtype_mapping = {
	"time_id": BigInteger(),
	"customer_id": BigInteger(),
	"courier_id": BigInteger(),
	"office_id": BigInteger(),
}

In [52]:
# Save the fact table to OLAP
service_fact.to_sql(
	"SERVICE_HOUR_FACT_TABLE", OLAP_connection, if_exists="replace", index=True, dtype=dtype_mapping
)

245